# Example: Cervical dystonia analysis

To illustrate how practical data analysis can be carried out in Jupyter notebooks, this short example will comprise three steps:

- Data import and processing with [Pandas](https://pandas.pydata.org)
- Visualizing the dataset with [Seaborn](https://seaborn.pydata.org)
- Building a statistical model with [PyMC3](http://docs.pymc.io)

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='notebook', style='ticks')

This dataset is from Table 6.9 of [Statistical Methods for the Analysis of Repeated Measurements](http://www.amazon.com/Statistical-Methods-Analysis-Repeated-Measurements/dp/0387953701) by Charles S. Davis, pp. 161-163 (Springer, 2002). These data are from a multicenter, randomized controlled trial of botulinum toxin type B (BotB) in patients with cervical dystonia from nine U.S. sites.

* Randomized to placebo (N=36), 5000 units of BotB (N=36), 10,000 units of BotB (N=37)
* Response variable: total score on Toronto Western Spasmodic Torticollis Rating Scale (TWSTRS), measuring severity, pain, and disability of cervical dystonia (high scores mean more impairment)
* TWSTRS measured at baseline (week 0) and weeks 2, 4, 8, 12, 16 after treatment began

## Data Import and Processing with Pandas

`pandas` is a Python package providing fast, flexible, and expressive data structures designed to work with relational or labeled data both. It is a fundamental high-level building block for doing practical, real world data analysis in Python.

Pandas makes it easy to get data in and out of your Python session. We can read in our data, in csv format, using the `read_csv` function.

In [ ]:
cdystonia = pd.read_csv("cdystonia.csv", index_col=None)
cdystonia.head()

`read_csv` returns the data in a pandas `DataFrame`, a tabular data structure, encapsulating multiple series like columns in a spreadsheet. Data are stored internally as a 2-dimensional object, but the `DataFrame` allows us to represent and manipulate higher-dimensional data.

This dataset includes repeated measurements of the same individuals (longitudinal data). Its possible to present such information in (at least) two ways: showing each repeated measurement in their own row, or in multiple columns representing multiple measurements.

## Pivoting

The `pivot` method allows a DataFrame to be transformed easily between long and wide formats in the same way as a pivot table is created in a spreadsheet. It takes three arguments: `index`, `columns` and `values`, corresponding to the DataFrame index (the row headers), columns and cell values, respectively.

For example, we may want the `twstrs` variable (the response variable) in wide format according to patient, as we saw with the unstacking method above:

In [ ]:
twstrs_wide = cdystonia.pivot(index='patient', columns='obs', values='twstrs')
twstrs_wide.head()

To convert the dataset to a "wide" format, we can extract the patient-level columns and merge them with the observations by patient above, so that we have one row per patient.

In [ ]:
cdystonia_wide = (cdystonia[['patient','site','id','treat','age','sex']]
                  .drop_duplicates()
                  .merge(twstrs_wide, right_index=True, left_on='patient', how='inner')
                  .head())
cdystonia_wide

A slightly cleaner way of doing this is to set the patient-level information as an index before unstacking:

In [ ]:
(cdystonia.set_index(['patient','site','id','treat','age','sex','week'])['twstrs']
     .unstack('week').head())

To convert our "wide" format back to long, we can use the `melt` function, appropriately parameterized. This function is useful for `DataFrame`s where one
or more columns are identifier variables (`id_vars`), with the remaining columns being measured variables (`value_vars`). The measured variables are "unpivoted" to
the row axis, leaving just two non-identifier columns, a *variable* and its corresponding *value*, which can both be renamed using optional arguments.

In [ ]:
pd.melt(cdystonia_wide, id_vars=['patient','site','id','treat','age','sex'], 
        var_name='obs', value_name='twsters').head()

This illustrates the two formats for longitudinal data: **long** and **wide** formats. Its typically better to store data in long format because additional data can be included as additional rows in the database, while wide format requires that the entire database schema be altered by adding columns to every row as data are collected.

The preferable format for analysis depends entirely on what is planned for the data, so it is imporant to be able to move easily between them.

## Data visualization with Seaborn

There is a variety of choices for data visualization in Python. Here, we will introduce Seaborn, a **high-level** interface for drawing attractive statistical graphics. Seaborn includes several attractive features:

- default themes that are aesthetically pleasing
- custom color palettes
- a variety of standard statistical plots that look great without manual customization

These features make Seaborn great for exploratory data analysis.

Notice that this data represents time series of individual patients, comprised of follow-up measurements at 2-4 week intervals following treatment. We can use the `pointplot` function from the `seaborn` package to visualize time series data.

Where `pointplot` is particularly useful is in representing the central tendency and variance of multiple replicate measurements. Having examined individual responses to treatment, we may now want to look at the average response among treatment groups. Where there are mutluple outcomes (y variable) for each predictor (x variable), `pointplot` will plot the mean, and calculate the 95% confidence interval for the mean, using bootstrapping:

In [ ]:
ordered_treat = ['Placebo', '5000U', '10000U']
g = sns.FacetGrid(cdystonia, col='treat', col_order=ordered_treat)
g.map(sns.pointplot, 'week', 'twstrs', color='0.5', order=None)

`displot` can also fit parametric data models (instead of a kde). For example, we may wish to fit the data to normal distributions. We can used the distributions included in the SciPy package; Seaborn knows how to use these distributions to generate a fit to the data.

In [ ]:
from scipy.stats import norm

g = sns.FacetGrid(cdystonia, row='treat', col='week')
g.map(sns.distplot, 'twstrs', kde=False, fit=norm)

## Data Analysis with PyMC3

PyMC3 is a Python package for Bayesian statistical modeling and Probabilistic Machine Learning focusing on advanced Markov chain Monte Carlo (MCMC) and variational inference (VI) algorithms. Its flexibility and extensibility make it applicable to a large suite of problems.

We will use PyMC3 to quickly build a generalized linear model (GLM) for analyzing the cervical dystonia dataset. To prepare for this, we will perform a couple of data transformations with Pandas to get the subset of data we need. 

First, one of the important predictor variables, the treatment levels, is a categorical variable. To transform this variable to a numeric type, we will create **indicator** variables, or **dummy** variables, that indicate the treatment level in one of three columns.

In [ ]:
treatment_ind = pd.get_dummies(cdystonia.treat)
treatment_ind.head()

We will use the two treatment levels, `10000U` and `5000U`, leaving `Placebo` as the baseline level.

Simiarly, gender is encoded as a text string. We will create an indicator for the female category, with male as the baseline.

In [ ]:
female = cdystonia.sex.replace({'F':1, 'M':0})

We can combine these indicator columns with the numeric predictor, `age`, and the outcome variable, `twstrs`, to create an analysis subset of the dataset. 

For simplicity, we will try to predict the outcome variable at the 4-week point of the study. In a more complete analysis, we would use all of the outcomes in a joint analysis.

In [ ]:
analysis_subset = (cdystonia[['age', 'twstrs']].join(female)
                                    .join(treatment_ind[['10000U', '5000U']]))[cdystonia.week==4]

analysis_subset.columns = ('age', 'outcome', 'female', 'high_dose', 'low_dose')

analysis_subset.head()

We can specify a GLM in just a couple of lines, using the `glm` module for automating the process. It takes a formula string that specifies a model in the form:

    outcome ~ predictors
    
This formula is passed to the `GLM` module using the `from_formula` function. This creates a Bayesian regression model, with reasonable default prior distributions for the variables.

In [ ]:
from pymc3 import Model, glm

with Model() as model:
    
    glm.GLM.from_formula('outcome ~ age + female + low_dose + high_dose', 
                         data=analysis_subset)
    

We will fit the model using Markov chain Monte Carlo (MCMC), which draws samples from the posterior distributions. We will generate 1000 samples after running the algorithm for 2000 iterations for tuning.

In [ ]:
from pymc3 import sample

with model:
    
    trace = sample(1000, tune=2000)

PyMC3 includes several functions for plotting the output from the analysis. For example, we can plot the posterior distributions of the two dose levels, which indicate the expected change in `twstrs` from baseline. The distribtion shows the posterior **uncertainty** from the analysis. We can plot a reference line for zero (i.e. no change).

In [ ]:
from pymc3 import plot_posterior

plot_posterior(trace, varnames=['low_dose', 'high_dose'], ref_val=0, color='LightSeaGreen');

Or, we can create a **forest plot** of all the covariates, which show their estimated value along with posterior probability intervals that characterize the uncertainty. The `R-hat` values are measures for monitoring convergence, which we want to be close to 1.

In [ ]:
from pymc3 import forestplot

forestplot(trace, varnames=['age', 'female', 'low_dose', 'high_dose']);